# Tutorial - build MNB with sklearn

This tutorial demonstrates how to use the Sci-kit Learn (sklearn) package to build Multinomial Naive Bayes model, rank features, and use the model for prediction. 

The data from the Kaggle Sentiment Analysis on Movie Review Competition are used in this tutorial. Check out the details of the data and the competition on Kaggle.
https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews

The tutorial also includes sample code to prepare your prediction result for submission to Kaggle. Although the competition is over, you can still submit your prediction to get an evaluation score.

# Step 1: Read in data

In [1]:
# read in the training data

# the data set includes four columns: PhraseId, SentenceId, Phrase, Sentiment
# In this data set a sentence is further split into phrases 
# in order to build a sentiment classification model
# that can not only predict sentiment of sentences but also shorter phrases

# A data example:
# PhraseId SentenceId Phrase Sentiment
# 1 1 A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .1

# the Phrase column includes the training examples
# the Sentiment column includes the training labels
# "0" for very negative
# "1" for negative
# "2" for neutral
# "3" for positive
# "4" for very positive


import pandas as p
train=p.read_csv("/Users/kenmckee/Desktop/GS/S18/tm/ks/train.tsv", delimiter='\t')
y=train['Sentiment'].values
X=train['Phrase'].values

# Step 2: Split train/test data for hold-out test

In [2]:
# check the sklearn documentation for train_test_split
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
# "test_size" : float, int, None, optional
# If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split. 
# If int, represents the absolute number of test samples. 
# If None, the value is set to the complement of the train size. 
# By default, the value is set to 0.25. The default will change in version 0.21. It will remain 0.25 only if train_size is unspecified, otherwise it will complement the specified train_size.    

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train[0])
print(y_train[0])
print(X_test[0])
print(y_test[0])

(93636,) (93636,) (62424,) (62424,)
almost in a class with that of Wilde
3
escape movie
2


# Step 2.1 Data Checking

In [3]:
# Check how many training examples in each category
# this is important to see whether the data set is balanced or skewed

#print(list(y_train))
print("train vectors")
training_labels = set(y_train)
print(training_labels)
from scipy.stats import itemfreq
training_category_dist = itemfreq(y_train)
print(training_category_dist)

print("\n test vectors")
training_labels = set(y_test)
print(training_labels)
from scipy.stats import itemfreq
training_category_dist = itemfreq(y_test)
print(training_category_dist)

train vectors
{0, 1, 2, 3, 4}
[[    0  4141]
 [    1 16449]
 [    2 47718]
 [    3 19859]
 [    4  5469]]

 test vectors
{0, 1, 2, 3, 4}
[[    0  2931]
 [    1 10824]
 [    2 31864]
 [    3 13068]
 [    4  3737]]


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  app.launch_new_instance()


# Exercise A

# Step 3: Vectorization

In [4]:
# sklearn contains two vectorizers

# CountVectorizer can give you Boolean or TF vectors
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

# TfidfVectorizer can give you TF or TFIDF vectors
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

# Read the sklearn documentation to understand all vectorization options

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# several commonly used vectorizer setting

#  unigram boolean vectorizer, set minimum document frequency to 5
unigram_bool_vectorizer = CountVectorizer(encoding='latin-1', binary=True, min_df=5, stop_words='english')

#  unigram term frequency vectorizer, set minimum document frequency to 5
unigram_count_vectorizer = CountVectorizer(encoding='latin-1', binary=False, min_df=5, stop_words='english')

#  unigram and bigram term frequency vectorizer, set minimum document frequency to 5
gram12_count_vectorizer = CountVectorizer(encoding='latin-1', ngram_range=(1,2), min_df=5, stop_words='english')

#  unigram tfidf vectorizer, set minimum document frequency to 5
unigram_tfidf_vectorizer = TfidfVectorizer(encoding='latin-1', use_idf=True, min_df=3, stop_words='english')

#  unigram SVM vectorizer, set minimum document frequency to 5


## Step 3.1: Vectorize the training data

In [5]:
# The vectorizer can do "fit" and "transform"
# fit is a process to collect unique tokens into the vocabulary
# transform is a process to convert each document to vector based on the vocabulary
# These two processes can be done together using fit_transform(), or used individually: fit() or transform()



#  unigram boolean vectorizer, set minimum document frequency to 5
X_train_bool = unigram_bool_vectorizer.fit_transform(X_train)
X_test_bool = unigram_bool_vectorizer.transform(X_test)

#  unigram term frequency vectorizer, set minimum document frequency to 5
X_train_uni = unigram_count_vectorizer.fit_transform(X_train) 
X_test_uni = unigram_count_vectorizer.transform(X_test) 

#  unigram and bigram term frequency vectorizer, set minimum document frequency to 5
X_train_gram = gram12_count_vectorizer.fit_transform(X_train) 
X_test_gram = gram12_count_vectorizer.transform(X_test)

#  unigram tfidf vectorizer, set minimum document frequency to 5
X_train_tfidf = unigram_tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = unigram_tfidf_vectorizer.transform(X_test)



# check the content of a document vector
print(X_train_tfidf.shape)
print(X_train_tfidf[0].toarray())

# check the size of the constructed vocabulary
print(len(unigram_tfidf_vectorizer.vocabulary_))

# print out the first 10 items in the vocabulary
print("\n tfidf")
print(list(unigram_tfidf_vectorizer.vocabulary_.items())[:10])
print("\n unigram")
print(list(unigram_count_vectorizer.vocabulary_.items())[:10])
print("\n gram12")
print(list(gram12_count_vectorizer.vocabulary_.items())[:10])
print("\n bool")
print(list(unigram_bool_vectorizer.vocabulary_.items())[:10])


# check word index in vocabulary
print(unigram_tfidf_vectorizer.vocabulary_.get('imaginative'))

(93636, 14113)
[[0. 0. 0. ... 0. 0. 0.]]
14113

 tfidf
[('class', 2207), ('wilde', 13849), ('derring', 3309), ('chilling', 2103), ('affecting', 363), ('meanspirited', 7770), ('personal', 9061), ('low', 7463), ('involved', 6654), ('worth', 13991)]

 unigram
[('class', 1858), ('wilde', 11742), ('derring', 2802), ('chilling', 1764), ('affecting', 313), ('meanspirited', 6557), ('personal', 7662), ('low', 6296), ('involved', 5602), ('worth', 11868)]

 gram12
[('class', 5020), ('wilde', 33787), ('derring', 7552), ('chilling', 4755), ('affecting', 825), ('meanspirited', 19199), ('personal', 22506), ('low', 18281), ('involved', 15905), ('worth', 34261)]

 bool
[('class', 1858), ('wilde', 11742), ('derring', 2802), ('chilling', 1764), ('affecting', 313), ('meanspirited', 6557), ('personal', 7662), ('low', 6296), ('involved', 5602), ('worth', 11868)]
6195


# Exercise B

# Step 4: Train a MNB classifier

In [6]:
# import the MNB module
from sklearn.naive_bayes import MultinomialNB

# initialize the MNB model
nb_clf= MultinomialNB()

# use the training data to train the MNB model
nb_clf.fit(X_train_gram,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

# Step 4.1 Interpret a trained MNB model

In [7]:
## interpreting naive Bayes models
## by consulting the sklearn documentation you can also find out how to print the coef_ for naive Bayes 
## which are the conditional probabilities
## http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

# the code below will print out the conditional prob of the word "worthless" in each category
# sample output
# -8.98942647599 -> logP('worthless'|very negative')
# -11.1864401922 -> logP('worthless'|negative')
# -12.3637684625 -> logP('worthless'|neutral')
# -11.9886066961 -> logP('worthless'|positive')
# -11.0504454621 -> logP('worthless'|very positive')
# the above output means the word feature "worthless" is indicating "very negative" 
# because P('worthless'|very negative) is the greatest among all conditional probs

gram12_count_vectorizer.vocabulary_.get('worthless')
for i in range(0,5):
  print(nb_clf.coef_[i][unigram_tfidf_vectorizer.vocabulary_.get('worthless')])

-11.17300930484645
-11.875865740663391
-10.916968513994783
-10.609291793489117
-11.275264544742841


Sample output:

-8.5389826392
-10.6436375867
-11.8419845779
-11.4778370023
-10.6297551464

In [8]:
# sort the conditional probability for category 0 "very negative"
# print the words with highest conditional probs
# these can be words popular in the "very negative" category alone, or words popular in all cateogires

feature_ranks = sorted(zip(nb_clf.coef_[0], gram12_count_vectorizer.get_feature_names()))
very_negative_features = feature_ranks[-10:]
print(very_negative_features)

[(-6.629714522576447, 'time'), (-6.61913241324591, 'characters'), (-6.608661113378615, 'minutes'), (-6.608661113378615, 'story'), (-6.598298326343068, 'comedy'), (-6.377218759249709, 'just'), (-5.825901774128982, 'like'), (-5.663620968218473, 'bad'), (-5.5205201245778, 'film'), (-5.0096945008118094, 'movie')]


In [9]:
# pretty print of top and bottom features

# This is a function I found from stackexchange, and adapted a little bit
# The purpose is to print the top and bottom features nicely
# https://stackoverflow.com/questions/11116697/how-to-get-most-informative-features-for-scikit-learn-classifiers

# You can find many useful scripts from stackexchange or GitHub
# Most tasks are not so unique, so someone in this world might have done something similar and shared their code

def show_most_and_least_informative_features(vectorizer, clf, class_idx=0, n=10):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[class_idx], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[-n:])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))
        


In [10]:
# show most positive features (category 4)
show_most_and_least_informative_features(gram12_count_vectorizer, nb_clf, class_idx=4, n=10)

	-11.2753	000            		-6.4631	performance    
	-11.2753	10 15          		-6.4155	comedy         
	-11.2753	10 minutes     		-6.3700	great          
	-11.2753	10 seconds     		-6.3408	story          
	-11.2753	10 year        		-6.2917	performances   
	-11.2753	10 years       		-6.0178	good           
	-11.2753	100 minute     		-5.8771	funny          
	-11.2753	101            		-5.7822	best           
	-11.2753	101 minutes    		-5.4060	movie          
	-11.2753	101 premise    		-4.8985	film           


In [11]:
nb_clf.coef_[4][0]

-11.275264544742841

# Exercise C

In [12]:
# calculate log ratio of conditional probs

# In this exercise you will calculate the log ratio 
# between conditional probs in the "very negative" category
# and conditional probs in the "very positive" category,
# and then sort and print out the top and bottom 10 words

# the conditional probs for the "very negative" category is stored in nb_clf.coef_[0]
# the conditional probs for the "very positive" category is stored in nb_clf.coef_[4]

# You can consult with similar code in week 4's sample script on feature weighting
# Note that in sklearn's MultinomialNB the conditional probs have been converted to log values.

# Your code starts here

ratios = (nb_clf.feature_log_prob_[0]/nb_clf.feature_log_prob_[4])
feature_ranks = sorted(zip(ratios,gram12_count_vectorizer.get_feature_names()))
botton_features = feature_ranks[0:10]
top_features = feature_ranks[-10:]
print("Top Features:")
[print(x[0],x[1]) for x in reversed(top_features) ]
print("Bottom Features:")
[print(x[0],x[1]) for x in botton_features]




# Your code ends here

Top Features:
1.6208651798490843 moving
1.5996164730633227 beautiful
1.5734688648152801 beautifully
1.55951982562155 powerful
1.5522968776555852 solid
1.5411232562154322 touching
1.5334311254747184 gorgeous
1.525529428444065 excellent
1.5025592326480928 best
1.4915094880043833 wonderful
Bottom Features:
0.5727210076533503 bad
0.5889360542577535 worst
0.6513698972838479 stupid
0.6615753633576739 worse
0.6706797310337252 contrived
0.6706797310337252 unfunny
0.6731097382481359 awful
0.6756082053000537 poorly
0.6808267524826445 waste
0.6837140127558956 minutes


[None, None, None, None, None, None, None, None, None, None]

# Step 5: Test the MNB classifier

In [13]:
# print confusion matrix (row: ground truth; col: prediction)

from sklearn.metrics import confusion_matrix
y_pred = nb_clf.fit(X_train_gram, y_train).predict(X_test_gram)
cm=confusion_matrix(y_test, y_pred, labels=[0,1,2,3,4])
print(cm)

[[  867  1253   725    69    17]
 [  786  4440  4943   609    46]
 [  459  2961 24437  3600   407]
 [   41   513  5082  6375  1057]
 [    6    46   602  1911  1172]]


In [14]:
# print classification report

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
print(precision_score(y_test, y_pred, average=None))
print(recall_score(y_test, y_pred, average=None))

from sklearn.metrics import classification_report
target_names = ['0','1','2','3','4']
print(classification_report(y_test, y_pred, target_names=target_names))

[0.4015748  0.48192771 0.68280757 0.5074021  0.4342349 ]
[0.29580348 0.41019956 0.76691564 0.48783287 0.31362055]
             precision    recall  f1-score   support

          0       0.40      0.30      0.34      2931
          1       0.48      0.41      0.44     10824
          2       0.68      0.77      0.72     31864
          3       0.51      0.49      0.50     13068
          4       0.43      0.31      0.36      3737

avg / total       0.58      0.60      0.59     62424



# Step 5.1 Interpret the prediction result

In [15]:
## find the calculated posterior probability
posterior_probs = nb_clf.predict_proba(X_test_vec)

## find the posterior probabilities for the first test example
print(posterior_probs[0])

# find the category prediction for the first test example
y_pred = nb_clf.predict(X_test_vec)
print(y_pred[0])

# check the actual label for the first test example
print(y_test[0])

NameError: name 'X_test_vec' is not defined

# Step 5.2 Error Analysis

In [ ]:
# print out specific type of error for further analysis

# print out the very positive examples that are mistakenly predicted as negative
# according to the confusion matrix, there should be 53 such examples
# note if you use a different vectorizer option, your result might be different

err_cnt = 0
for i in range(0, len(y_test)):
    if(y_test[i]==4 and y_pred[i]==1):
        print(X_test[i])
        err_cnt = err_cnt+1
print("errors:", err_cnt)

# Exercise D

In [ ]:
# Can you find linguistic patterns in the above errors? 
# What kind of very positive examples were mistakenly predicted as negative?

# Can you write code to print out the errors that very negative examples were mistakenly predicted as very positive?
# Can you find lingustic patterns for this kind of errors?
# Based on the above error analysis, what suggestions would you give to improve the current model?

# Your code starts here
err_cnt = 0
for i in range(0, len(y_test)):
    if(y_test[i]==4 and y_pred[i]==0):
        print(X_test[i])
        err_cnt = err_cnt+1
print("errors:", err_cnt)
# Your code ends here

# Step 6: write the prediction output to file

In [ ]:
y_pred=nb_clf.predict(X_test_vec)
output = open('/Users/byu/Desktop/data/prediction_output.csv', 'w')
for x, value in enumerate(y_pred):
  output.write(str(value) + '\n') 
output.close()

# Step 6.1 Prepare submission to Kaggle sentiment classification competition

In [ ]:
########## submit to Kaggle submission

# we are still using the model trained on 60% of the training data
# you can re-train the model on the entire data set 
#   and use the new model to predict the Kaggle test data
# below is sample code for using a trained model to predict Kaggle test data 
#    and format the prediction output for Kaggle submission

# read in the test data
kaggle_test=p.read_csv("/Users/byu/Desktop/data/kaggle/test.tsv", delimiter='\t') 

# preserve the id column of the test examples
kaggle_ids=kaggle_test['PhraseId'].values

# read in the text content of the examples
kaggle_X_test=kaggle_test['Phrase'].values

# vectorize the test examples using the vocabulary fitted from the 60% training data
kaggle_X_test_vec=unigram_tfidf_vectorizer.transform(kaggle_X_test)

# predict using the NB classifier that we built
kaggle_pred=nb_clf.fit(X_train_vec, y_train).predict(kaggle_X_test_vec)

# combine the test example ids with their predictions
kaggle_submission=zip(kaggle_ids, kaggle_pred)

# prepare output file
outf=open('/Users/byu/Desktop/data/kaggle/kaggle_submission.csv', 'w')

# write header
outf.write('PhraseId,Sentiment\n')

# write predictions with ids to the output file
for x, value in enumerate(kaggle_submission): outf.write(str(value[0]) + ',' + str(value[1]) + '\n')

# close the output file
outf.close()

# Exercise E

In [ ]:
# generate your Kaggle submissions with boolean representation and TF representation
# read in the test data
kaggle_test=p.read_csv("/Users/kenmckee/Desktop/GS/S18/tm/ks/test.tsv", delimiter='\t') 

# preserve the id column of the test examples
kaggle_ids=kaggle_test['PhraseId'].values

# read in the text content of the examples
kaggle_X_test=kaggle_test['Phrase'].values

# vectorize the test examples using the vocabulary fitted from the 60% training data
kaggle_X_test_vec=unigram_tfidf_vectorizer.transform(kaggle_X_test)

# predict using the NB classifier that we built
kaggle_pred=nb_clf.fit(X_train_vec, y_train).predict(kaggle_X_test_vec)

# combine the test example ids with their predictions
kaggle_submission=zip(kaggle_ids, kaggle_pred)

# prepare output file
outf=open('/Users/kenmckee/Desktop/GS/S18/tm/ks/kaggle_submission.csv', 'w')

# write header
outf.write('PhraseId,Sentiment\n')

# write predictions with ids to the output file
for x, value in enumerate(kaggle_submission): outf.write(str(value[0]) + ',' + str(value[1]) + '\n')

# close the output file
outf.close()
# submit to Kaggle

# report your scores here
# which model gave better performance in the hold-out test
# which model gave better performance in the Kaggle test

Sample output:

(93636, 9968)
[[0 0 0 ..., 0 0 0]]
9968
[('disloc', 2484), ('surgeon', 8554), ('camaraderi', 1341), ('sketchiest', 7943), ('dedic', 2244), ('impud', 4376), ('adopt', 245), ('worker', 9850), ('buy', 1298), ('systemat', 8623)]
245

# BernoulliNB

In [ ]:
from sklearn.naive_bayes import BernoulliNB
X_train_vec_bool = unigram_tfidf_vectorizer.fit_transform(X_train)
bernoulliNB_clf = BernoulliNB(X_train_vec_bool, y_train)

# Cross Validation

In [ ]:
# cross validation

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
nb_clf_pipe = Pipeline([('vect', CountVectorizer(encoding='latin-1', binary=False)),('nb', MultinomialNB())])
scores = cross_val_score(nb_clf_pipe, X, y, cv=3)
avg=sum(scores)/len(scores)
print(avg)

# Exercise F

In [ ]:
# run 3-fold cross validation to compare the performance of 
# (1) BernoulliNB (2) MultinomialNB with TF vectors (3) MultinomialNB with boolean vectors

# Your code starts here
# cross validation

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
nb_clf_pipe = Pipeline([('vect', CountVectorizer(encoding='latin-1', binary=False)),('nb', MultinomialNB())])
scores = cross_val_score(nb_clf_pipe, X, y, cv=3)
avg=sum(scores)/len(scores)
print(avg)

# Your code ends here

# Optional: use external linguistic resources such as stemmer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk.stem

english_stemmer = nltk.stem.SnowballStemmer('english')
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([english_stemmer.stem(w) for w in analyzer(doc)])

stem_vectorizer = StemmedCountVectorizer(min_df=3, analyzer="word")
X_train_stem_vec = stem_vectorizer.fit_transform(X_train)

In [ ]:
# check the content of a document vector
print(X_train_stem_vec.shape)
print(X_train_stem_vec[0].toarray())

# check the size of the constructed vocabulary
print(len(stem_vectorizer.vocabulary_))

# print out the first 10 items in the vocabulary
print(list(stem_vectorizer.vocabulary_.items())[:10])

# check word index in vocabulary
print(stem_vectorizer.vocabulary_.get('adopt'))